In [1]:
from astropy.io import fits
from astropy import wcs, units
import numpy as np
import pylab as plt 
#from spectral_cube import SpectralCube
from astropy.coordinates import SkyCoord
#import aplpy
import time
%matplotlib inline


In [51]:
cand  = np.loadtxt('/home/themba/Documents/MSc/Thesis/catalogue/ppmos9/ppmos9.txt') #Loads text file with candidates 
X,Y,Z = cand[:,1], cand[:,2], cand[:,3]                                                 #coordinates.
size  = cand.shape[0]


hdu = fits.open('/home/themba/Documents/MSc/Thesis/Data/ppmos1/wsrt/ppmosALL_9_kms.fits')
Header = hdu[0].header
cube = hdu[0].data
#data = hdu[0].data
#voxel_units =  h['BUNIT'], vel_units = h['CUNIT3']
std = np.mean(cube)
dv = Header['CDELT3']   #channel width [km/s]

dir = '/home/themba/Documents/MSc/Thesis/catalogue/ppmos9/Update/' #directory to save data products.
dir1 = '/media/themba/c18d84e6-53eb-44c6-be34-18234c64ea16/MSc/ppmos9/Results/linewidths/'
#boundaries of the cube 
zzl,yyl,xxl = 0.,0.,0.0
zzu,yyu,xxu = cube.shape 
s,v     = 50,100 

In [52]:
def cublets(z,y,x,k):            #This function extracts subcubes from the origion dataset.
    ##cublet boundaries    
    Xl,Xu,Yl,Yu,Zl,Zu=0,0,0,0,0,0
    #X-axis boundaries
    ss=0
    Sxu,Sxl = x+s, x - s #sub_cube upper & lower bounds, respectively
    Syu,Syl = y+s, y - s
    Szu,Szl = z+v, z - v
    
    
    if (Syu <= yyu and Syl >= yyl and Sxu <= xxu and Sxl >= xxl):  # This is for a subcube
                                                            #that is contain within the cube
        Yu  = Syu
        Yl  = Syl
        Xu  = Sxu
        Xl  = Sxl   
    else:
    
        if Sxu > xxu:
            Sxu = xxu - x #half width
            Xu  = x+Sxu
            Xl  = x-Sxu
            
            Yu  = y + Sxu
            Yl  = y - Sxu
        if Sxl < xxl:
            Sxl = x-xxl
            Xu  = x+Sxl
            Xl  = x-Sxl
            
            Yu  = y + Sxl
            Yl  = y - Sxl

        #Y-axis boundaries
        if Syu > yyu:
            Syu = yyu - y
            Yu  = y+Syu
            Yl  = y-Syu
            
            Xu  = x + Syu
            Xl  = x - Syu
            
        if Syl < yyl:
            Syl = y-yyl
            Yu  = y+Syl
            Yl  = y-Syl

            Xu  = x + Syl
            Xl  = x - Syl        
        
    #Z-axis boundaries
    if Szu > zzu:    
        Szu = zzu
    if Szl < zzl:    
        Szl = zzl   
            
    #if (Szu <= zzu and Szl >= zzl):    
    #   Zu = Szu    
    #    Zl = Szl  
    if Szu <=zzu:
        Szu=Szu
    if Szl>=zzl:
        Szl=Szl
    #print Szl,Szu
            
    sub_cube1 = cube[Szl:Szu,Yl:Yu,Xl:Xu] #cublet
    fits.writeto(dir1+'P_'+str(int(X[k]))+':'+str(int(Y[k]))+':'+str(int(Z[k]))+'.fits',
                 sub_cube1,overwrite=True) #unmasked 

In [53]:
#header.update({'NAXIS1': 100, 'NAXIS2': 100})
Header.set('ctype3', 'velo')
w= wcs.WCS(Header)

CDELT1 = Header.get('CDELT1')
CTYPE1 = Header.get('CTYPE1')
CROTA1 = Header.get('CROTA1')
CUNIT1 = Header.get('CUNIT1')
 
CDELT2 = Header.get('CDELT2')
CTYPE2 = Header.get('CTYPE2')
CROTA2 = Header.get('CROTA2')
CUNIT2 = Header.get('CUNIT2') 

CDELT3 = Header.get('CDELT3')
CTYPE3 = Header.get('CTYPE3')
CROTA3 = Header.get('CROTA3')
CUNIT3 = Header.get('CUNIT3')

BUNIT  = Header.get('BUNIT')
BLANK  = Header.get('BLANK')
EPOCH  = Header.get('EPOCH')
FREQ0  = Header.get('FREQ0')

def headr(cube,x,y,z,t):
    #naxis3,naxis2,naxis1 = cube.shape

    
    #hdu    = fits.open(cube)
    data = fits.getdata(cube)
    header  = fits.getheader(cube)
    dim = data.shape
    
    crval1,crval2,crval3 = w.all_pix2world(x,y,z,0)
    zz,yy,xx = data.shape
    
    header.insert('EXTEND ', ('CDELT1', CDELT1), after=True)
    header.insert('CDELT1 ', ('CROTA1', CROTA1), after=True)
    header.insert('CROTA1 ', ('CRPIX1', 0.5*xx), after=True)
    header.insert('CRPIX1 ', ('CRVAL1', crval1[()]), after=True)
    header.insert('CRVAL1 ', ('CTYPE1', CTYPE1), after=True)
    header.insert('CTYPE1 ', ('CUNIT1', CUNIT1), after=True)
    
    header.insert('CUNIT1 ', ('CDELT2', CDELT2), after=True)
    header.insert('CDELT2 ', ('CROTA2', CROTA2), after=True)
    header.insert('CROTA2 ', ('CRPIX2', 0.5*yy), after=True)
    header.insert('CRPIX2 ', ('CRVAL2', crval2[()]), after=True)
    header.insert('CRVAL2 ', ('CTYPE2', CTYPE2), after=True)
    header.insert('CTYPE2 ', ('CUNIT2', CUNIT2), after=True)    
    header['BPA']=0.11
    header['BMAJ']=6.4913E-03
    header['BMIN']=4.6136E-03
    
    if t==1:
        header.insert('CUNIT2 ', ('CDELT3', CDELT3*1e3), after=True)
        header.insert('CDELT3 ', ('CROTA3', CROTA3), after=True)
        header.insert('CROTA3 ', ('CRPIX3', 0.5*zz), after=True)
        header.insert('CRPIX3 ', ('CRVAL3', crval3[()]*1e3), after=True)
        header.insert('CRVAL3 ', ('CTYPE3', "VELO-HEL"), after=True)
        header.insert('CTYPE3 ', ('CUNIT3', "m/s"), after=True)
           
        header.insert('CUNIT3 ', ('BLANK', BLANK), after=True)
        header.insert('BLANK ', ('BUNIT', BUNIT), after=True) 
        header.insert('BUNIT ', ('EPOCH', EPOCH), after=True)
        header.insert('EPOCH ', ('RESTFREQ', FREQ0), after=True)
        fits.writeto(filename=cube,data=data,header=header,clobber=True) 
    elif t==0:
        header.insert('CUNIT2 ', ('BUNIT', 'Jy/beam m/s'), after=True)
        header.insert('BUNIT ', ('EPOCH', EPOCH), after=True)
        header.insert('EPOCH ', ('RESTFREQ', FREQ0), after=True)
        fits.writeto(filename=cube,data=data,header=header,overwrite=True)

In [54]:
for i in range(size): #creating subcubes
    cublets(Z[i],Y[i],X[i],i)

/home/themba/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:66: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [55]:
#Updatigng headers
for k in range(size):
    headr(dir1+'P_'+str(int(X[k]))+':'+str(int(Y[k]))+':'+str(int(Z[k]))+'.fits',
          X[k],Y[k],Z[k],1) #cube header